In [134]:
import osmnx as ox
import pyvista as pv
import networkx as nx
import numpy as np
import random, heapq
from functools import lru_cache
from collections import OrderedDict

# Create a networkx graph from a place
place = "Chandigarh, India"
G = ox.graph_from_place(place, network_type="drive")

In [135]:
nodes, edges = ox.graph_to_gdfs(G)
lines = []
# convert each edge into a line for graph
for _, row in edges.iterrows():
    x_pts = row['geometry'].xy[0]
    y_pts = row['geometry'].xy[1]
    z_pts = np.zeros(len(x_pts))
    pts = np.column_stack((x_pts, y_pts, z_pts))
    line = pv.lines_from_points(pts)
    lines.append(line)

In [126]:
combined_lines = lines[0].merge(lines[1:])
# combined_lines.plot(line_width=1.5, cpos='xy')

In [13]:
def get_start_goal_nodes():
    global G
    nodes = list(G.nodes)

    # Use two random nodes as start and goal (you can replace them with specific nodes if you want)
    start = random.choice(nodes)
    goal = random.choice(nodes)

    # Ensure start and goal nodes are not the same
    while start == goal:
        goal = random.choice(nodes)

    return start, goal


@lru_cache(maxsize=None)  # Add caching to the successors function
def successors(node):
    global G

    # Find the neighbors of the current node
    neighbors = list(G.neighbors(node))

    # Calculate the edge weight and add it as a tuple (neighbor, weight)
    successors = [(neighbor, G[node][neighbor][0]['length'])
                  for neighbor in neighbors]

    return successors


@lru_cache(maxsize=None)  # Add caching to the great_circle_distance function
def great_circle_distance(p1, p2):
    global G
    coord1 = G.nodes[p1]['y'], G.nodes[p1]['x']
    coord2 = G.nodes[p2]['y'], G.nodes[p2]['x']
    return ox.distance.great_circle(*coord1, *coord2)

In [206]:

def dijkstra(start, goal):
    # Initialize variables
    # Distances from start to each node
    distances = {node: float('inf') for node in G.nodes}
    distances[start] = 0  # Distance to start is 0
    visited = set()  # Set of visited nodes
    queue = [(0, start)]  # Priority queue (distance, node)
    predecessors = {node: None for node in G.nodes}

    while queue:
        current_distance, current_node = heapq.heappop(queue)

        if current_node == goal:
            return distances[goal], reconstruct_path(start, goal, predecessors), visited

        if current_node in visited:
            continue

        visited.add(current_node)

        for neighbor, weight in successors(current_node):
            new_distance = current_distance + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                predecessors[neighbor] = current_node
                heapq.heappush(queue, (new_distance, neighbor))

    return float('inf'), []  # No path found


def reconstruct_path(start, goal, predecessors):
    path = [goal]
    while path[-1] != start:
        path.append(predecessors[path[-1]])
    path.reverse()
    return path


start, goal = get_start_goal_nodes()

print(f"{start}, {goal}")

dijkstra_cost, dijkstra_path, visited_nodes = dijkstra(start, goal)
print("Dijkstra Result:")
if dijkstra_cost != float('inf'):
    print(f"Path: {dijkstra_path}")
    print(f"Cost: {dijkstra_cost}\n")
else:
    print("Dijkstra: No solution available for given start and goal\n")

1428213113, 1431135549
Dijkstra Result:
Path: [1428213113, 1428213109, 1428213212, 1428213222, 1428213250, 1428213295, 1428213301, 1428213340, 1428213420, 1428213464, 1428213470, 1428213490, 1428213494, 1428213542, 1428213560, 1428213590, 1428213586, 1429260167, 1429141339, 1429141349, 1429260385, 1429260422, 1429260892, 1429260912, 1429260916, 1429260950, 6473425816, 1429141397, 1429141405, 1429141408, 1429141411, 1429141417, 10166264951, 1429261108, 1429261124, 1429261140, 1429261171, 1429261190, 1429261205, 1429261215, 6883935450, 1429261244, 1429261272, 1429261281, 1429261308, 1429261410, 1429261424, 1429261513, 1429261579, 1429261647, 1429261689, 1429261736, 1429380148, 1429141563, 1429141566, 1429141585, 1429141607, 1429262257, 10230398020, 1429141813, 1429141821, 1429141850, 1429141888, 10653480131, 1429142110, 1429078345, 1429078403, 1429078407, 1429078445, 1429078473, 1429078549, 1429078587, 9444013025, 1429078911, 1429078939, 1429078945, 1429078956, 1429078969, 1429078999, 14

In [207]:
len(visited_nodes), len(nodes)

(9011, 12262)

In [208]:
G.subgraph(dijkstra_path)

In [209]:
G.subgraph(visited_nodes)

In [211]:
nodes, edges = ox.graph_to_gdfs(G.subgraph(visited_nodes))
visited_lines = []
# convert each edge into a line for graph
for _, row in edges.iterrows():
    x_pts = row['geometry'].xy[0]
    y_pts = row['geometry'].xy[1]
    z_pts = np.zeros(len(x_pts))
    pts = np.column_stack((x_pts, y_pts, 0.000001))
    line = pv.lines_from_points(pts)
    visited_lines.append(line)

In [212]:
nodes, edges = ox.graph_to_gdfs(G.subgraph(G.subgraph(dijkstra_path)))
path_lines = []
# convert each edge into a line for graph
for _, row in edges.iterrows():
    x_pts = row['geometry'].xy[0]
    y_pts = row['geometry'].xy[1]
    z_pts = np.zeros(len(x_pts))
    pts = np.column_stack((x_pts, y_pts, 0.000002))
    line = pv.lines_from_points(pts)
    path_lines.append(line)

In [1]:
# Plot the elevation map
plotter = pv.Plotter()
plotter.add_mesh(lines[0].merge(lines[1:]), line_width=1,
                 color='blue', label='Road Network')
plotter.add_mesh(visited_lines[0].merge(visited_lines[1:]), line_width=2,
                 color='green', label='Visited Nodes')
plotter.add_mesh(path_lines[0].merge(path_lines[1:]), line_width=3,
                 color='red', label='Path', pbr=True)

plotter.show(cpos='xy')

NameError: name 'pv' is not defined